In [ ]:
# These are just defaults will be overwritten if you use https://github.com/yaananth/nimport
su="tfs-weu-3"
start="2019-07-16T13:30:00.0000000Z"
end="2019-07-16T13:44:00.0000000Z"
url="https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl="https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"
service="tfs"

In [ ]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd

from pandas import Series, DataFrame

from datetime import datetime, timedelta, timezone

from urllib.parse import urlencode, quote_plus

from requests.utils import requote_uri

import time

import numpy as np

from matplotlib import pyplot as plt

from nimport.utils import tokenize

import json
import os

In [ ]:
params = {
    "su": su,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl,
    "service": service
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
    

In [ ]:
# This isn't needed if you are bootstraping
#!pip install Kqlmagic --no-cache-dir --upgrade
#!pip install nimport --no-cache-dir --upgrade

In [ ]:
# kusto/python utilities
def getTime(timestamp, d):
    return int((time.mktime(getDateTime(timestamp).timetuple()) + (d * 60)) * 1000);
                
def getDateTime(timestamp):
    s = timestamp[:23] + 'Z' # only allow 5 decimals of precision
    for f in ("%Y-%m-%d %H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%S.%fZ"):
        try:
            return datetime.strptime(s, f);
        except:
            pass
           
def getKustoQuery(csl_filename, params):

    return tokenize(os.path.join(root, 'queries', csl_filename), params)
                
states = {
    "ja": "ja",
    "healthagent": "healthagent"
};

state = "";

In [ ]:
%load_ext Kqlmagic

In [ ]:
%kql AzureDataExplorer://tenant="Microsoft.com";code;cluster='VSO';database='VSO'

In [ ]:
# LocationName.csl
q_loc = getKustoQuery("LocationName.csl", params)
locationNameResult = %kql -query q_loc
locationName = locationNameResult[0]['Tenant']
params["locationName"] = locationName

In [ ]:
# WhatChanged.csl
q_whatChanged = getKustoQuery("WhatChanged.csl", params)
q_whatChanged_r = %kql -query q_whatChanged
q_whatChanged_df = q_whatChanged_r.to_dataframe();

In [ ]:
q_whatChanged_df

In [ ]:
print('=' * 50)
print('Report!')
print('=' * 50, '\n\n')

jarvisParams = {'su': su, 'start': getTime(start, -10), 'end': getTime(end, 10), 'service': service }

# jarvis
jarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/DevOpsReports/TFS DevOpsReports""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='RoleInstance']","key":"value","replacement":""},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
print('Jarvis dashboard link:\n', requote_uri(jarvisLink), '\n')

# what changed? analysis
print()
print('What changed? =============================')
if(len(q_whatChanged_df.index) == 0):
    print("No relevant changes found...")
else:
    up_prefix = "";
    mit_prefix = "";
    text = "";
    for index, row in q_whatChanged_df.iterrows():
        if(row.title.lower().find('upgrade') != -1):
            if not up_prefix:
                up_prefix += "Looks like, there's upgrade in progress...\n\n";
            text += """%s %s %s \n""" % (row.TIMESTAMP, row.title, row.buildNumber);
        if(row.title.lower().find('mitigation') != -1):
            if not mit_prefix:
                mit_prefix += "Looks like, there are some mitigations by health agent...\n\n";
                state += states["healthagent"];
            text += """%s %s %s""" % (row.TIMESTAMP, row.title, row.buildNumber);
            
    if text:
        print(up_prefix + mit_prefix + text)
    else:
        print(q_whatChanged_df)